# Betweenness Centrality
_Betweenness Centrality_ is a measure of centrality in a graph based on calculating geodesic (shortest) paths between nodes.
There is at least one shortest path between every pair of nodes in a connected graph.
The shortest path can be based on the number of relationships that the path passes through in an unweighted network or the sum of the weights of the relationships in a weighted network.

The betweenness centrality for each node is the number of these shortest paths that pass through that node.
The nodes that most frequently lie on these shortest paths are will have a higher betweenness centrality score.


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

In [3]:
create_graph_query = '''
CREATE (nAlice:User {id:'Alice'})
,(nBridget:User {id:'Bridget'})
,(nCharles:User {id:'Charles'})
,(nDoug:User {id:'Doug'})
,(nMark:User {id:'Mark'})
,(nMichael:User {id:'Michael'})
CREATE (nAlice)-[:MANAGE]->(nBridget)
,(nAlice)-[:MANAGE]->(nCharles)
,(nAlice)-[:MANAGE]->(nDoug)
,(nMark)-[:MANAGE]->(nAlice)
,(nCharles)-[:MANAGE]->(nMichael);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 5, 'nodes-created': 6, 'properties-set': 6}


In [4]:
streaming_query = """
CALL algo.betweenness.stream('User','MANAGE',{direction:'out'}) 
YIELD nodeId, centrality 
RETURN nodeId,centrality order by centrality desc limit 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,nodeId,centrality
0,140,4.0
1,142,2.0
2,8,0.0
3,9,0.0
4,10,0.0
5,11,0.0
6,12,0.0
7,13,0.0
8,20,0.0
9,21,0.0
